In [5]:
import os
os.chdir('/home/justine/Nextcloud/Cired/Recherche/Econometrie/fail_to_board_probability/f2b_code')

In [2]:
from f2b.f2b_estimation.data import Data
from f2b.f2b_estimation.likelihood import Likelihood
from f2b.postprocessing.output_analysis import load_estimated_f2b, get_estimation_results_and_indicators

In [3]:
origin_station = "VIN"
destination_stations = ["NAT", "LYO", "CHL", "AUB", "ETO", "DEF"]
date = "04/02/2020"
direction = 2

In [7]:
f2b_estimated = load_estimated_f2b(origin_station, "line_search_refacto")
data = Data(date, origin_station, destination_stations, direction)
likelihood = Likelihood(data, f2b_estimated)

Trips construction...


100%|██████████| 8336/8336 [00:01<00:00, 4969.33it/s]


Runs construction...


100%|██████████| 317/317 [00:00<00:00, 3265.26it/s]


(Offline) Compute access and egress individual likelihood contributions...


100%|██████████| 8336/8336 [00:03<00:00, 2515.69it/s]


In [23]:
hessian = likelihood.get_global_log_likelihood_hessian()
print(hessian.shape)

(317, 317)


In [11]:
list_of_empty_runs = []
for (run_position, run_code) in enumerate(data.runs_chronological_order):
    if not data.runs[run_code].associated_trips:
        list_of_empty_runs.append((run_position, run_code))
list_of_empty_runs

[(296, '      '),
 (304, 'ZUMO99'),
 (306, 'ZURE01'),
 (308, 'ZURE03'),
 (309, 'TAPI01'),
 (310, 'ZURE05'),
 (311, 'VIDA03'),
 (312, 'ZEBU07'),
 (313, 'ZEBU09'),
 (314, 'TEDI07'),
 (315, 'VIRO09'),
 (316, 'YVAN69')]

In [24]:
from numpy import delete
successive_indices_to_delete = [list_of_empty_runs[i][0] - i for i in range(len(list_of_empty_runs))]

for run_position in successive_indices_to_delete:
    hessian = delete(hessian, run_position, 0)
    hessian = delete(hessian, run_position, 1)
    print(run_position)
    print(hessian.shape)


296
(316, 316)
303
(315, 315)
304
(314, 314)
305
(313, 313)
305
(312, 312)
305
(311, 311)
305
(310, 310)
305
(309, 309)
305
(308, 308)
305
(307, 307)
305
(306, 306)
305
(305, 305)


In [25]:
from numpy import linalg
fischer_info = -hessian
asymptotic_variance = linalg.inv(fischer_info)

In [28]:
gradient = likelihood.get_global_log_likelihood_gradient()